### Install AI Core Python SDK

In [ ]:
!pip install ai-api-client-sdk

In [1]:
import json
import requests
import time
import yaml
from IPython.display import clear_output
from pprint import pprint

from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
from ai_api_client_sdk.models.status import Status
from ai_api_client_sdk.models.target_status import TargetStatus
from ai_api_client_sdk.models.parameter_binding import ParameterBinding

In [2]:
# Setup
aic_service_key_path = "./resources/aic_service_key.json"
git_setup_file_path = "./resources/git_setup.json"
docker_secret_file_path = "./resources/docker_secret.json"
env_file_path = "./resources/env.json"
resource_group = "default"
serving_workflow_file = "./scenario/proxy.yaml"
connection_name = "default"
path_prefix = "app"

### 1. Connect to your AI Core instance

In [3]:
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

# AI API client that talks to the AI Core instance.
ai_api_client = AIAPIV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key["clientid"],
    client_secret = aic_service_key["clientsecret"]
)

### 2. Onboard the Git repository that contains the templates

In [ ]:
with open(git_setup_file_path) as gs:
		setup_json = json.load(gs)

repo_json = setup_json["repo"]

response = ai_api_client.rest_client.post(
		path="/admin/repositories",
		body={
				"name": repo_json["name"],
				"url": repo_json["url"],
				"username": repo_json["username"],
				"password": repo_json["password"]
		}
)
print(response)

### 3. Register an application

In [ ]:
app_json = setup_json["app"]
response = ai_api_client.rest_client.post(
		path="/admin/applications",
		body={
				"applicationName": app_json["applicationName"],
				"repositoryUrl": app_json["repositoryUrl"],
				"revision": app_json["revision"],
				"path": app_json["path"]
		}
)

### 4 Docker Hub (optional)
#### 4.1 Register Docker secret on SAP BTP, AI Core (optional)

In [ ]:
with open(docker_secret_file_path) as dsf:
    docker_secret = json.load(dsf)

pprint(docker_secret)

response = ai_api_client.rest_client.post(
    path="/admin/dockerRegistrySecrets",
    body={
        "name": docker_secret["name"],
        "data": docker_secret["data"]
    }
)
print(response)

#### 4.2 Build and push Docker image (optional)
```
$ cd proxy
$ docker build -t {DOCKER_USERNAME}/azure-openai-proxy .
$ docker push {DOCKER_USERNAME}/azure-openai-proxy
```

THROUGH THE DOCKER CLI.  
See: https://developers.sap.com/tutorials/ai-core-aiapi-clientsdk-workflows.html#f824a41d-efe8-4883-8238-caef4ac5f789

### 5. Create a resource group

In [ ]:
ai_api_client.rest_client.post(
    path="/admin/resourceGroups",
    body={
        "resourceGroupId": resource_group
    }
)

### 6. Create configuration to serve the model

In [ ]:
# Create another AI API client to use different base url.
ai_api_v2_client = AIAPIV2Client(
    base_url=aic_service_key["serviceurls"]["AI_API_URL"] + "/v2/lm",
    auth_url=aic_service_key["url"] + "/oauth/token",
    client_id=aic_service_key["clientid"],
    client_secret=aic_service_key["clientsecret"],
    resource_group=resource_group)

with open(serving_workflow_file) as swf:
    serving_workflow = yaml.safe_load(swf)

scenario_id = serving_workflow["metadata"]["labels"]["scenarios.ai.sap.com/id"]
executable_name = serving_workflow["metadata"]["name"]

with open(env_file_path) as efp:
    environment_values = json.load(efp)

parameter_bindings = [ParameterBinding(key=key, value=value) for key, value in environment_values.items()]


serve_configuration = {
    "name": f"{resource_group}-serve",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": parameter_bindings,
    "input_artifact_bindings": []
}

serve_config_resp = ai_api_v2_client.configuration.create(**serve_configuration)

assert serve_config_resp.message == "Configuration created"

pprint(vars(serve_config_resp))
print("configuration for serving the model created")

### 7. Actually serve the proxy

In [ ]:
deployment_resp = ai_api_v2_client.deployment.create(serve_config_resp.id)
pprint(vars(deployment_resp))

# Poll deployment status.
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print("...... deployment status ......", flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")

# Allow some time for deployment URL to get ready.
time.sleep(10)

### 8. Do an inference request

In [ ]:
endpoint = f"{deployment.deployment_url}/v2/envs"
headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           "ai-resource-group": resource_group,
           "Content-Type": "application/json"}
response = requests.get(endpoint, headers=headers)

legacy_davinci = False # set True if you have a davinci model deployment on Azure OpenAI Services
if legacy_davinci:
    body = {
        "engine": "<YOUR ENGINE>", # include your davinci engine from a deployment of an Azure OpenAI services model
        "prompt": "Classify the following news article into 1 of the following categories: categories: [Business, Tech, Politics, Sport, Entertainment]\n\nnews article: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective:\n\nClassified category:",
        "max_tokens": 60,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "top_p": 1,
        "best_of": 1,
        "stop": "null"
    }
    endpoint = f"{deployment.deployment_url}/v2/completion"
else:
    body = {
        "engine": "<YOUR ENGINE>", # include your engine from a deployment of an Azure OpenAI services model
        "prompt": "Classify the following news article into 1 of the following categories: categories: [Business, Tech, Politics, Sport, Entertainment]\n\nnews article: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective:\n\nClassified category:",
        "max_tokens": 60,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stop": "null"
    }
    endpoint = f"{deployment.deployment_url}/v2/chat-completion"

headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           "ai-resource-group": resource_group,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=body)

print("Inference result:", response.json())
pprint(vars(response))

### 9. Kill deployment (optional)

In [ ]:
delete_resp = ai_api_v2_client.deployment.modify(deployment_resp.id,
                                                 target_status=TargetStatus.STOPPED)
status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print("...... killing deployment ......", flush=True)
    print(f"Deployment status: {deployment.status}")